# In this notebook, we are going to cover some of the most fundamentals concepts of tensors using TensorFlow

# More specifically, we are going to cover:

- Introduction to tensors
- Getting information from tensors
- Manipulating tensors
- Tensors * Numpy
- Using @tf.function
- Using GPUs with TensorFlow (or TFUs)
- Exercises to try myself

## Introduction to Tensors

In [ ]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

In [ ]:
# Check the dimensions of our vector
vector.ndim

In [ ]:
# Create a matrix (has more than one dimension)
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

In [ ]:
# Check the dimensions of our matrix
matrix.ndim

In [ ]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameters
another_matrix

In [ ]:
# What is the number dimensions of another_matrix?
another_matrix.ndim

In [ ]:
# Let's create a tensor
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                       [10,11,12]],
                      [[13,14,15],
                       [16,17,18]]])
tensor

In [ ]:
tensor.ndim

## What we are created so far:

- Scalar: a single number
- Vector: a number with direction (e.g. wind speed and direction)
- Matrix: a 2-dimensional array of numbers
- Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional is a scalar, a 1-dimensional is a vector)

## Creating tensors with tf.Variable

In [ ]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

In [ ]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

In [ ]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

In [ ]:
# Let's try change our unchangeable tensor
unchangeable_tensor[0].assign(7)

## Creating random tensors

Random tensors are tensors of some abitrary size which contain random numbers

In [ ]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
# Are they equal?
random_1, random_2, random_1 == random_2

## Shuffle the order of elements in a tensor

In [ ]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't affect learning)
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [2,5]])
# Shuffle our non-shuffle
tf.random.shuffle(not_shuffled)

In [ ]:
# Shuffle our non-shuffle
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

In [ ]:
not_shuffled

## Other ways to make tensors

In [ ]:
# Create a tensor of all one
tf.ones([10,7])

In [ ]:
# Create a tensor of all zeroes
tf.zeros(shape=(3,4))

## Turn Numpy arrays into tensors

The main difference between Numpy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical compiting).

In [ ]:
# You can also turn Numpy arrays into tensors
import numpy as np

numpy_A = np.arange(1, 25, dtype=np.int32) # Create a Numpy array between 1 and 25
numpy_A
# x = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

In [ ]:
A = tf.constant(numpy_A, shape=(3,8))
B = tf.constant(numpy_A)
A, B

In [ ]:
A.ndim

## Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
- Shape
- Rank
- Axis or dimension
- Size

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor

In [ ]:
rank_4_tensor[0]

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

In [ ]:
# Get various attributes of our tensor
print("Datatype of every elements: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

## Indexing tensors

Tensors can be indexed just like Python lists

In [ ]:
# Get the first 2 elements of each dimensions
rank_4_tensor[:2, :2, :2, :2]

In [ ]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

In [ ]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10,7], [3,4]])
rank_2_tensor

In [ ]:
# Get the last item of each of row of our rank 2 tensor
rank_2_tensor[:, -1]

In [ ]:
# Add in extra dimension  to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = -1) # "-1" means expand the final axis

In [ ]:
# Expand the 0-axis
tf.expand_dims(rank_2_tensor, axis=0) 

In [ ]:
rank_2_tensor

## Manipulating tensors (tensors operations)

**Basic operations**:
- `+`, `-`, `*`, `/`

In [ ]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10,7], [3,4]])
tensor + 10

In [ ]:
# Original tensor is unchanged 
tensor = tensor + 10
tensor

In [ ]:
# Multiplication also works
tensor * 10

In [ ]:
# Subtraction if you want
tensor - 10

In [ ]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10)

In [ ]:
tensor

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

There are two rules our tensors (or matrices) need to fulfill if we're going to matrix multiply them:
1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [ ]:
# Matrix multiplication in tensorflow
print(tensor)
tf.matmul(tensor, tensor)

In [ ]:
# Matrix multiplication in Python with operator "@"
tensor @ tensor

In [ ]:
# Create a tensor (3,2)
X = tf.constant([[1,2],
                 [3,4],
                 [5,6]])
# Create another tensor (2,3) tensor
Y = tf.constant([[7,8], [9, 10],[ 11, 12]])
X, Y

In [ ]:
# Try to matrix multiply tensors of same shape matrix
tf.matmul(X,Y)

In [ ]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2,3))

In [ ]:
X.shape, tf.reshape(Y, shape=(2,3)).shape

In [ ]:
# Try to matrix multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2,3))

In [ ]:
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

In [ ]:
tf.reshape(X, shape=(2,3)).shape, Y.shape

In [ ]:
# Try change the shape of X instead of Y
tf.matmul(tf.reshape(X, shape=(2,3)), Y)

In [ ]:
# Can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2,3))

In [ ]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

**The dot product**

Matrix  multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [ ]:
X, Y

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

In [ ]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

In [ ]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

In [ ]:
# Check the values of Y, reshaped Y and transposed Y
print("Normal Y: ")
print(Y, "\n")

print("Y reshaped to (2, 3): ")
print(tf.reshape(Y, shape=(2,3)), "\n")

print("Y transposed: ")
print(tf.transpose(Y))

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to get satisfy the matrix multiplication rules

In [ ]:
### Changing the datatype of a tensor

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

In [ ]:
C = tf.constant([7,10])
C.dtype